In [1]:
from pathlib import Path
import pandas as pd

In [2]:
data_path = Path('/home/akio/data/google-smartphone-decimeter-challenge/')

In [3]:
test_base = pd.read_csv(
    data_path / 'baseline_locations_test.csv'
)
sub = pd.read_csv(
    data_path / 'sample_submission.csv'
)

In [4]:
test_base.head()

,collectionName,phoneName,millisSinceGpsEpoch,latDeg,lngDeg,heightAboveWgs84EllipsoidM,phone
0,2020-05-15-US-MTV-1,Pixel4,1273608785432,37.416628,-122.082053,-30.69,2020-05-15-US-MTV-1_Pixel4
1,2020-05-15-US-MTV-1,Pixel4,1273608786432,37.416646,-122.082040,-31.76,2020-05-15-US-MTV-1_Pixel4
2,2020-05-15-US-MTV-1,Pixel4,1273608787432,37.416652,-122.082039,-31.65,2020-05-15-US-MTV-1_Pixel4
3,2020-05-15-US-MTV-1,Pixel4,1273608788432,37.416607,-122.082063,-31.52,2020-05-15-US-MTV-1_Pixel4
4,2020-05-15-US-MTV-1,Pixel4,1273608789432,37.416609,-122.082073,-28.95,2020-05-15-US-MTV-1_Pixel4


In [5]:
sub.head()

,phone,millisSinceGpsEpoch,latDeg,lngDeg
0,2020-05-15-US-MTV-1_Pixel4,1273608785432,37.904611,-86.481078
1,2020-05-15-US-MTV-1_Pixel4,1273608786432,37.904611,-86.481078
2,2020-05-15-US-MTV-1_Pixel4,1273608787432,37.904611,-86.481078
3,2020-05-15-US-MTV-1_Pixel4,1273608788432,37.904611,-86.481078
4,2020-05-15-US-MTV-1_Pixel4,1273608789432,37.904611,-86.481078


In [11]:
sub.assign(
    latDeg = test_base.latDeg,
    lngDeg = test_base.lngDeg
).to_csv('submission.csv',index=False)

In [12]:
sub.head()

,phone,millisSinceGpsEpoch,latDeg,lngDeg
0,2020-05-15-US-MTV-1_Pixel4,1273608785432,37.904611,-86.481078
1,2020-05-15-US-MTV-1_Pixel4,1273608786432,37.904611,-86.481078
2,2020-05-15-US-MTV-1_Pixel4,1273608787432,37.904611,-86.481078
3,2020-05-15-US-MTV-1_Pixel4,1273608788432,37.904611,-86.481078
4,2020-05-15-US-MTV-1_Pixel4,1273608789432,37.904611,-86.481078


In [13]:
import numpy as np 
from tqdm.notebook import tqdm

In [21]:
truths = (data_path / 'train').rglob('ground_truth.csv')

In [22]:

df_list = []
cols = ['collectionName','phoneName','millisSinceGpsEpoch','latDeg','lngDeg']

for t in tqdm(truths,total=73):
    df_phone = pd.read_csv(t,usecols=cols)
    df_list.append(df_phone)

df_truth = pd.concat(df_list,ignore_index=True)

df_basepreds = pd.read_csv(data_path / 'baseline_locations_train.csv',usecols=cols)

  0%|          | 0/73 [00:00<?, ?it/s]

In [27]:
df_all = df_truth.merge(df_basepreds, how='inner', on=cols[:3], suffixes=('_truth', '_basepred'))

In [28]:
df_all.head()

,collectionName,phoneName,millisSinceGpsEpoch,latDeg_truth,lngDeg_truth,latDeg_basepred,lngDeg_basepred
0,2020-05-29-US-MTV-2,Pixel4XL,1274830058447,37.523273,-122.354286,37.523279,-122.354322
1,2020-05-29-US-MTV-2,Pixel4XL,1274830059447,37.523273,-122.354286,37.523294,-122.354297
2,2020-05-29-US-MTV-2,Pixel4XL,1274830060447,37.523273,-122.354286,37.523292,-122.354307
3,2020-05-29-US-MTV-2,Pixel4XL,1274830061447,37.523273,-122.354286,37.523278,-122.354304
4,2020-05-29-US-MTV-2,Pixel4XL,1274830062447,37.523273,-122.354286,37.523281,-122.354304


In [35]:
def calc_haversine(lat1, lon1, lat2, lon2):
    """Calculates the great circle distance between two points
    on the earth. Inputs are array-like and specified in decimal degrees.
    """
    RADIUS = 6_367_000
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    dist = 2 * RADIUS * np.arcsin(a**0.5)
    return dist

In [36]:
df_all['dist'] = calc_haversine(df_all.latDeg_truth, df_all.lngDeg_truth, 
    df_all.latDeg_basepred, df_all.lngDeg_basepred)

In [37]:
df_all.dist.mean()


3.8468483749952074